# CinvesPy 
## Nov 2017

**Mariana Jaber** *mariana.ifunam@gmail.com*

# Estimación de parámetros


En esta libreta vamos a resolver un problema de optimización numérica. Queremos hallar los valores numéricos para los parámetros de un modelo al ajustarlo con datos observacionales.

Los parámetros libres son el valor de la llamada "constante de Hubble", $H_0$ y la fracción de materia al día de hoy, $\Omega_m$.


Usaremos datos de lo que se conoce como "Cronómetros Cósmicos" y se pueden halla en la referencia arXiv:1604.00183. Ahí se reportan valores para la *función de Hubble*, $H(z)$, a diferentes valores de $z$ con sus respectivos errores $\Delta H(z)$.



-------- · -------- · -------- · -------- · -------- · -------- · -------- · -------- · -------- · -------- · --------

Los pasos que seguiremos son: 


i. Usaremos *np.loadtxt* (ver la Lecture de **Numpy**) para cargar una tabla con los datos observacionales. 

ii. Con la función  *plt.errorbar* de Matplotlib haremos la gráfica de nuestros puntos observados, para tener una idea de cómo lucen nuestros datos.

iii.  Definir una función (ver libreta de **Introducción a Python**) para escribir nuestro *modelo*. 

iv. Definiremos la función que mide la "bondad del ajuste",  $\chi^2$. Usando el método  *differential_evolution* de *scipy.Optimize* para hallar el mínimo global de nuestra función. El valor de los parámetros que hagan $\chi^2=min$ serán los "mejores valores del ajuste".  

v. Usando lo que aprendimos en la charla de  **Multiprocessing** evaluaremos la función $\chi^2$ en una grid m-dimensional (con m, el número de parámetros describiendo nuestro modelo). 

vi. Por último, usaremos el formato conocido como *hdf5* para guardar nuestra malla y los valores de $\chi^2$ en cada punto y con la función *contourf* de Matplotlib  graficaremos los llamados "contornos de confianza" a (1, 2, y 3-$\sigma$).


In [ ]:
%pylab inline

## Paso 1.  Datos
Importamos nuestros puntos observacionales para $H(z)$


In [ ]:
dataHz = np.loadtxt('Hz_all.dat')

In [ ]:
dataHz.shape

In [ ]:
redshifts = dataHz[:,0]
obs = dataHz[:,1]
errors = dataHz[:,2]

Si deseamos ver cómo lucen nuestros datos, usaremos *errorbar* de **Matplotlib**


In [ ]:
figsize = (8, 6)                        # definimos el tamaño de nuestra figura
dpi = 300                                # dots per inch

rcParams['font.size'] = 15               # establecemos el tamaño de la fuente
rcParams['lines.linewidth'] = 2          # el grosor de las líneas
rcParams['mathtext.fontset'] = 'cm'      # y el tipo de fuente en LaTex 

fig1 = figure(figsize=figsize, dpi=dpi)  # definimos la figura

 
plt.errorbar(redshifts, obs, errors,      #plt.errorbar(x, y, error_y)
             xerr=None, 
             color='purple', marker='o', ls='None', 
             elinewidth =2, capsize=5, capthick = 1, 
             label='$Datos$')


plt.legend(loc='best', frameon=False)     #pon la legenda donde mejor quede sin marco

xlabel(r'$z$')                            
ylabel(r'$H(z) [km/s Mpc^{-1}]$')         


#savefig('data_plot.pdf', bbox_inches='tight') #guardar la salida como un archivo pdf, eps, jpg, png

## Paso 2. Modelo

Comenzamos por definir nuestro modelo.

### El Modelo: $\Lambda$CDM

El marco teórico mínimo en cosmología es asumir un Universo geométricamente plano y dominado por una *Constante Cosmológica*, $\Lambda$ al día de hoy.

Esto se parametriza de la siguiente forma:

Que el Universo sea plano implica que al día de hoy se cumple  $\Omega_m + \Omega_{DE} = 1$

Y que el causante de la Aceleración en la expansión sea una Constante Cosmológica, se puede representar con $w=-1$. 

La condición de "planitud" implica $\Omega_{DE} = 1- \Omega_m$.

Así que tendremos como parámetro libre el valor $\Omega_m$.

Adicionalmente, el valor al día de hoy de la constante de Hubble, $H_0$ quedará como parámetro libre del modelo.


La extensión más simple de este modelo sería considerar que el parámetro $w$, sea constante aunque no sea igual a -1, $w=cte\neq -1$. En esta extensión a nuestro modelito tendríamos 3 parámetros libres: $w$, $\Omega_m$, y $H_0$.

Para los propósitos de esta libreta, usaremos  $\Lambda$CDM como modelo, *i.e.*, $w=-1$ y tendremos solamente 2 parametros libres.




### $H(z) = H_0 \sqrt{\Omega_m(1+z)^3+\Omega_{\Lambda}}$



In [ ]:
def hubblefunc(z, w, H0, OmegaM):
    '''
    This function calculates the theoretical value for the Hubble function,
    this is, the expansion rate of the Universe in terms of redshift
    
    w: Dark Energy Equation of State. w=-1 for a Cosmological Constant
    H0: present value of Hubble constant
    OmegaM: fractional matter density
    
    '''
    matter_contribution = OmegaM *(1 + z)**3
    
    DE_contribution = (1 - OmegaM) * (1 + z)**(3 * (1 + w))

    Ez = np.sqrt(matter_contribution + DE_contribution)
    
    return H0*Ez

Hemos escrito nuestra función de tal manera que podamos fácilmente añadir el parámetro $w$ para una cosmología diferente a $\Lambda$CDM

In [ ]:
hubblefunc?

Si lo deseamos podemos evaluarla para valores particulares de sus parámetros.

In [ ]:
H1=hubblefunc(0.2, -1, 70, 0.3)
H2=hubblefunc(0.2, -1, 60, 0.3) 
H3=hubblefunc(0.2, -1, 70, 0.25) 
H1, H2, H3

Ahora podemos ver cómo nuestra función ajusta a los datos.



In [ ]:
figsize = (8, 6)                        
dpi = 300                                

rcParams['font.size'] = 12               
rcParams['lines.linewidth'] = 2          
rcParams['mathtext.fontset'] = 'cm'      

fig = figure(figsize=figsize, dpi=dpi)  

#### data ########
plt.errorbar(redshifts, obs, errors,      
             xerr=None, 
             color='purple', marker='o', ls='None', 
             elinewidth =2, capsize=5, capthick = 1, 
             label='$Datos$')

#### model ########
zarray = np.linspace(0.01,2.3,300)
Hubblefunc = np.vectorize(hubblefunc, excluded=(1, 2, 3))


plot(zarray, Hubblefunc(zarray, -1, 65, 0.3), color = 'r',  label=r'$model$: $H_0$=65, $\Omega_m$=0.3')
plot(zarray, Hubblefunc(zarray, -1, 75, 0.3), color = 'b', label=r'$model$:  $H_0$=75, $\Omega_m$=0.3')
plot(zarray, Hubblefunc(zarray, -1, 75, 0.25), color = 'black', label=r'$model$: $H_0$=75, $\Omega_m$=0.25')
plt.title(r'$\Lambda CDM$: $w=-1$')


plt.legend(loc='best', frameon=False)     

xlabel(r'$z$')                            
ylabel(r'$H(z) [km/s Mpc^{-1}]$')         


#savefig('data-model_plot.pdf', bbox_inches='tight') 


## Paso 3. 
## $\chi^2$ function

Ahora queremos definir la función $\chi^2$ dado nuestro modelo y los datos que tenemos.


Dado que los datos no están correlacionados, escribimos:



### $\chi^2(\vec{\theta}) = \sum_i^{N=26}\frac{\left[H(z_i)^{obs} - H(z_i, w=-1, H_0,\Omega_m)\right]^2}{\sigma_i^2}$

donde $\vec{\theta}$ es el vector de parámetros, $\vec{\theta}=\{H_0, \Omega_m\}$

In [ ]:
def chi_2(H0, OmegaM):
    '''
    Chi2 function for our H(z) data and LCDM model
    '''
    chi_sum=0
    w = -1 # we have set this to the LCDM scenario
    for (zi, obs, err) in dataHz:
        
        chi_sum += ((obs - hubblefunc(zi, w, H0, OmegaM) )/ err) ** 2
    
    return chi_sum

In [ ]:
chi_2(85, 0.25)

## Paso 4.
### Estimar los parámetros del mejor ajuste 

Para hallar el mejor valor de $H_0$ y $\Omega_m$. 

Para ello, necesitamos hallar el punto, $\vec{\theta}_{best}$ en espacio de parámetros tal que $\chi^2(\vec{\theta}_{best})$ sea mínimo. 






In [ ]:
from scipy.optimize import differential_evolution

In [ ]:
def chi2_minimization():

    bounds = [(10, 100), (0.1, 1)] 
    
    def func_to_minimize(x):

        H0, OmM = x #for LCDM

        val = chi_2(H0, OmM)
        ##print("Point:", x, "Function value:", val)
        return val

    result = differential_evolution(func_to_minimize,
                                    bounds=bounds
                                    #, disp=True)
                                    )
                                    


    chi2 = result.fun
    
    H0, OmM  = result.x #for LCDM
   
    print(result.x)
  

    full_result = np.array(
        
        [chi2, H0, OmM]
    )


    filetxt = 'bfv_result_LCDM.txt' #for LCDM
    
    np.savetxt(filetxt, full_result)



In [ ]:
chi2_minimization()


## Paso 5. Evaluar una función en una malla usando Multiprocessing

Queremos evaluar la función $\chi^2$ para diferentes valores de nuestros parámetros libres, $\vec{\theta}$. 

In [ ]:
from multiprocessing import Pool
from h5py import File

In [ ]:
def chi2_kernel(data):
    H0, OmegaM = data

    chi = chi_2(H0, OmegaM)

    return chi

In [ ]:
def parallel_chi2(kernel, data_array, processes=None):
    """
    Evaluates the chi2 function over the data saved in 'data_array'
    and distributes the workload among several independent python
    processes.
    """

    # Let's create a pool of processes to execute calculate chi2 in
    # parallel.

    
    with Pool(processes=processes) as pool:
        # The data accepted by the map method must be an iterable, like
        # a list, a tuple or a numpy array. The function is applied over
        # each element of the iterable. The result is another list with
        # the values returned by the function after being evaluated.
        #
        # [a, b, c, ...]  -> [f(a), f(b), f(c), ...]
        #
        # Here we use the imap method, so we need to create a list to
        # gather the results.
        results = []
        pool_imap = pool.imap(kernel, data_array)
        
        for result in pool_imap:
            results.append(result)
            
    return np.array(results)

# Reference: arXiv:1306.0573

In [ ]:
def exec_chi2():
    
    points = 200 
    m_params = 2
    
    H0_range = np.linspace(50, 100, points)
    OmegaMatter_range = np.linspace(0.1, 0.6, points)

    #this will create the coordinate axis for the array
    H, OM = meshgrid(H0_range, OmegaMatter_range, indexing='ij')

    #and we use them to create our grid
    grid_data = stack((H,OM), axis=-1)

    # Flatten array
    
    flattened_grid = grid_data.reshape(points * points, m_params)
    
    # Execute parallel routine
    chi2_data = parallel_chi2(chi2_kernel, flattened_grid)

    # Reshape chi2 results with a similar shape as the original
    # data_grid, but with only one element.¡: the chi squared.
    
    chi2_data = chi2_data.reshape(points, points, 1)
    
    #################
    # Save data to file
    
    filename = 'H0-OmegaM_grid.h5'
    
    with File(filename, 'w') as file:
        # Add data
        file['/DataGrid'] = grid_data
        file['/Chi2'] = chi2_data
        # Save data to file
        file.flush()

    print(chi2_data.shape)


In [ ]:
exec_chi2()

## Paso 6. Hacer la gráfica de contorno



In [ ]:

grid_file = 'H0-OmM_grid.h5'


In [ ]:
with File(grid_file, 'r') as datafile:
    data = datafile['/DataGrid'].value
    chi2data = datafile['/Chi2'].value
    
    H = data[:,:,0]
    Omega = data[:,:,1]
    Chi2 = chi2data[:,:,0]
    
    plt.contourf(H, Omega, Chi2)
    colorbar()
    xlabel(r'$H_0$')
    ylabel(r'$\Omega_m$')
    


In [ ]:
from IPython.display import Image

### Estadística. Regiones de confianza

Fuente: **Numerical Recipes in C**

In [ ]:
figure = Image(filename='./../fig_2.png', width=600)
figure

In [ ]:
bfv_file = 'bfv_result_LCDM.txt'

chi2min, Hbest, Ombest = np.loadtxt(bfv_file)

In [ ]:
d1s, d2s, d3s = 2.3, 6.17, 9.21 #(68.3%, 95.4%, 99%)

levels = chi2min, chi2min+d1s, chi2min+d2s, chi2min+d3s


                              
with File(grid_file, 'r') as datafile:
    data = datafile['/DataGrid'].value
    chi2data = datafile['/Chi2'].value
    H = data[:,:,0]
    Omega = data[:,:,1]
    Chi2 = chi2data[:,:,0]
    
    
    rcParams['font.size'] = 15               
    rcParams['lines.linewidth'] = 2          
    rcParams['mathtext.fontset'] = 'cm'      
    C = plt.contourf(H, Omega, Chi2, 
                 levels=levels,
                 #cmap=plt.cm.jet,
                 colors=('mediumblue', 'cornflowerblue', 'skyblue'),
                 alpha=0.7,
                 
                )
    plt.colorbar()
    
    ylabel(r'$\Omega_m$')
    xlabel(r'$H_0[km/sMpc]$')
    xlim(55, 80)         


savefig('contour_plot_sigmas.pdf', bbox_inches='tight') 





    
    